In [302]:
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.feature_selection import SelectFromModel

In [245]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.info()
print('-'*40)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null

In [254]:
combined[train_df.Fare.notnull()].groupby(['Pclass']).median().reset_index()[['Pclass', 'Fare']]

/home/akai/virtualenvs/tensor_flow/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Pclass,Fare
0,1,60.0000
1,2,15.0458
2,3,8.0500


In [324]:
random_state = 42
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
combine = [train_df, test_df]
target = train_df['Survived']
test_ids = test_df['PassengerId']
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    dataset['Title'] = dataset['Title'].replace(['Jonkheer', 'Don', 'Sir', 'Countess', 'Lady', 'Dona'], 'Royalty')
    dataset['Title'] = dataset['Title'].replace(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer')
    dataset['Title'] = dataset['Title'].replace(['Mlle', 'Ms'], 'Miss')
    dataset['Title'] = dataset['Title'].replace(['Mme'], 'Mrs')
    dataset['Title'] = dataset['Title'].astype('category')
    dataset['Title'] = dataset['Title'].cat.codes

combined = pd.concat([train_df, test_df])
grouped_median_age = combined[train_df.Age.notnull()].groupby(['Sex', 'Pclass', 'Title']).median()
grouped_median_age = grouped_median_age.reset_index()[['Sex', 'Pclass', 'Title', 'Age']]
grouped_median_age

def assign_age(row):
    condition = (
        (grouped_median_age['Sex'] == row['Sex']) & 
        (grouped_median_age['Title'] == row['Title']) & 
        (grouped_median_age['Pclass'] == row['Pclass'])
    )
    return grouped_median_age[condition]['Age'].values[0]

for dataset in combine:
    dataset['Age'] = dataset.apply(lambda row: assign_age(row) if np.isnan(row['Age']) else row['Age'], axis=1)
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
    dataset.loc[dataset.Embarked.isnull(), 'Embarked'] = 'S'
    dataset.loc[dataset.Cabin.notnull(), 'Cabin'] = dataset.loc[dataset.Cabin.notnull(), 'Cabin'].map(lambda x: x[0])
    dataset.loc[dataset.Cabin.isnull(), 'Cabin'] = 'U' #unknown
    
def cleanTicket(ticket):
    ticket = ticket.replace('.', '')
    ticket = ticket.replace('/', '')
    ticket = ticket.split()
    ticket = map(lambda t : t.strip(), ticket)
    ticket = list(filter(lambda t : not t.isdigit(), ticket))
    if len(ticket) > 0:
        return ticket[0]
    else: 
        return 'XXX'
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'AgeBand'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'AgeBand'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'AgeBand'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'AgeBand'] = 3
    dataset.loc[ dataset['Age'] > 64, 'AgeBand'] = 4
    dataset['AgeBand'] = dataset['AgeBand'].astype(int)
    dataset.loc[dataset['Fare'].isnull(), 'Fare'] = 8
    dataset.loc[dataset['Fare'] <= 8, 'FareBand'] = 0
    dataset.loc[(dataset['Fare'] > 8) & (dataset['Fare'] <= 14.454), 'FareBand'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'FareBand']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'FareBand'] = 3
    dataset['FareBand'] = dataset['FareBand'].astype(int)
    dataset['Ticket'] = dataset['Ticket'].map(cleanTicket)
    for column in ['Sex', 'Cabin', 'Embarked', 'Ticket']:
        dataset[column] = dataset[column].astype('category')
        dataset[column] = dataset[column].cat.codes

for dataset in combine:
    dataset.drop(['PassengerId', 'Name', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize'], axis=1, inplace=True)
y = train_df['Survived']
train_df.drop(['Survived'], axis=1, inplace=True)
print(train_df.head())
## array([0.81564246, 0.80446927, 0.80898876, 0.79213483, 0.8079096 ])
# clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, min_samples_leaf=100, max_depth=2, random_state=random_state)
# array([0.79888268, 0.81564246, 0.81460674, 0.80337079, 0.85875706])
clf = RandomForestClassifier(max_depth=4, max_features=3, n_estimators=400)
# clf = VotingClassifier(
#     estimators=[
#         ('lr', LogisticRegression()),
#         ('svc', SVC()),
#         ('tree', DecisionTreeClassifier(max_depth=1)),
#     ],
#     voting='hard',
# )
# clf = AdaBoostClassifier(
#     DecisionTreeClassifier(max_depth=1), n_estimators=500,
#     algorithm='SAMME', learning_rate=0.5
# )
# clf = DecisionTreeClassifier()
# cv = cross_val_score(clf, train_df, y, cv=5)
# param_test = {'n_estimators': np.linspace(100,500,10, dtype=np.int),
#              'max_depth': [1,2,3,4],
#              'max_features': [2,3,4,5]}
# gs = GridSearchCV(estimator=clf, param_grid=param_test, cv=5, n_jobs=5, verbose=1)
# gs.fit(train_df, y)
# gs.best_params_, gs.best_score_
clf.fit(train_df, y)
cv = cross_val_score(clf, train_df, y, cv=5)
cv
# features = pd.DataFrame()
# features['feature'] = train_df.columns
# features['importance'] = clf.feature_importances_
# features.sort_values(by=['importance'], ascending=True, inplace=True)
# features.set_index('feature', inplace=True)
# features.plot(kind='barh', figsize=(25, 25))
# model = SelectFromModel(clf, prefit=True)
# train_reduced = model.transform(train_df)
# print(train_reduced.shape)

/home/akai/virtualenvs/tensor_flow/lib/python3.5/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()
/home/akai/virtualenvs/tensor_flow/lib/python3.5/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


   Pclass  Sex  Ticket  Cabin  Embarked  Title  IsAlone  AgeBand  FareBand
0       3    1       1      8         2      2        0        1         0
1       1    0      10      2         0      3        0        2         3
2       3    0      26      8         2      1        1        1         0
3       1    0      30      2         2      3        0        2         3
4       3    1      30      8         2      2        1        2         1


array([0.81005587, 0.79329609, 0.8258427 , 0.78089888, 0.84745763])

In [322]:
gs.best_params_, gs.best_score_

({'max_depth': 4, 'max_features': 5, 'n_estimators': 277}, 0.8226711560044894)

In [296]:
forest = RandomForestClassifier()
parameter_grid = {
    'max_depth' : [4, 6, 8],
    'n_estimators': [50, 10],
    'max_features': ['sqrt', 'auto', 'log2'],
    'min_samples_split': [2, 3, 10],
    'min_samples_leaf': [1, 3, 10],
    'bootstrap': [True, False],
}
grid_search = GridSearchCV(forest,
    scoring='accuracy',
    param_grid=parameter_grid,
    cv=5,
    verbose=1
)
grid_search.fit(train_df, y)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


[Parallel(n_jobs=1)]: Done 1620 out of 1620 | elapsed:   60.0s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [50, 10], 'max_depth': [4, 6, 8], 'min_samples_leaf': [1, 3, 10], 'max_features': ['sqrt', 'auto', 'log2'], 'bootstrap': [True, False], 'min_samples_split': [2, 3, 10]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=1)

In [297]:
grid_search.best_score_

0.8260381593714927

In [327]:
y_pred = clf.predict(test_df)
submission = pd.DataFrame({
    "PassengerId": test_ids,
    "Survived": y_pred
})
submission.to_csv('titanic_result_feature_engineer.csv')

In [ ]:
! kaggle competitions submit titanic -f titanic_result_feature_engineer.csv -m "some extensive feature engineer"